# Realtor API Walkthrough

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera, arielherrera@analyticsariel.com                                                      |
| External References   | <a href="https://rapidapi.com/apidojo/api/realtor?endpoint=apiendpoint_e259775d-d98e-479f-8440-206d6d4fa892" target="_blank">Realtor API</a>|
| Input Datasets        | List for Sale |
| Output Datasets       | Table    |
| Input Data Source     | API |
| Output Data Source    | Pandas Dataframe |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 13th May 2020 | Ariel Herrera | Notebook created to get data on properties for sale. |

## Other Details
This Notebook is a prototype.

In [2]:
import pandas as pd
import requests

## Functions

In [4]:
def get_api_key(api_key_id = "Realtor"):
  """
  Get the api key for website accessing.

  Table of key type and key value for privacy.

  Parameters
  ----------
  @api_key_id [string]: Key value in dataframe

  Returns
  -------
  [string]: API Key

  """
  # load api keys file
  df_api_keys = spark.read.format('csv').options(header='true', inferSchema='true').load('FileStore/tables/api_keys.csv').toPandas()
  
  # return api key if in dataset
  try:
    # get api key from id
    api_key = df_api_keys.loc[df_api_keys['Id'] == api_key_id]['Key'].iloc[0] # get key by id
    # return api key
    return api_key
  except IndexError:
    # get api key id list
    api_key_id_list = df_api_keys['Id'].unique().tolist()
    # print error message
    print('Cannot map key. Api key id must be one of the following options {0}'.format(api_key_id_list))

In [5]:
def api_property_list_for_sale(api_key, city, state, prop_type, limit=200):
  # url for api
  url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

  # enter parameters
  querystring = {
    "sort":"relevance",
    "city":city,
    "offset":"0",
    "limit":limit,
    "state_code":state,
    "prop_type":prop_type
  }

  # header
  headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': api_key
  }

  # response
  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json() # json format

In [6]:
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

## Variables

In [8]:
# api key to accessd ata
realtor_api_key = get_api_key(api_key_id = "Realtor")
city = "Newark"
state = "NJ"
prop_type = "single_family"

## Data: API Property List for Sale

In [10]:
property_list_for_sale_response = api_property_list_for_sale(api_key=realtor_api_key, 
                                                             city=city, 
                                                             state=state, 
                                                             prop_type=prop_type,
                                                             limit=200)

In [11]:
property_list_for_sale_response

Out[14]: {'meta': {'build': '3.23.91',
 'schema': 'core.3',
 'tracking_params': {'channel': 'for_sale',
 'siteSection': 'for_sale',
 'city': 'Newark',
 'county': 'unknown',
 'neighborhood': 'unknown',
 'searchCityState': 'Newark, NJ',
 'state': 'NJ',
 'zip': 'unknown',
 'srpPropertyStatus': 'srp:for_sale',
 'listingActivity': 'unknown',
 'propertyStatus': 'for_sale',
 'propertyType': 'single_family',
 'searchBathrooms': 'any',
 'searchBedrooms': 'any',
 'searchMaxPrice': 'unknown',
 'searchMinPrice': 'unknown',
 'searchRadius': 'unknown',
 'searchHouseSqft': 'any',
 'searchLotSqft': 'any',
 'searchResults': '127',
 'sortResults': 'relevance',
 'searchCoordinates': 'unknown',
 'version': '1.0'},
 'tracking': 'type|meta|data|resource_type|property_list|query|client_id|rdc_mobile_native,13.3.0.53|prop_status|for_sale|schema|core.3|limit|offset|city|Newark|state_code|NJ|sort|relevance|prop_type|single_family|count|total^5K|0|3J|3J^^$0|1|2|$3|4|5|$6|7|8|9|A|B|C|O|D|P|E|F|G|H|I|J|K|L]|8|9|M|Q|N|R]]',
 'returned_rows': 127,
 'matching_rows': 127},
 'properties': [{'property_id': 'M5185136719',
 'listing_id': '2916184785',
 'rdc_web_url': 'https://www.realtor.com/realestateandhomes-detail/273-Lake-St_Newark_NJ_07104_M51851-36719',
 'prop_type': 'single_family',
 'address': {'city': 'Newark',
 'line': '273 Lake St',
 'postal_code': '07104',
 'state_code': 'NJ',
 'state': 'New Jersey',
 'county': 'Essex',
 'fips_code': '34013',
 'lat': 40.760086,
 'lon': -74.178922,
 'neighborhood_name': 'Seventh Avenue'},
 'branding': {'listing_office': {'list_item': {'name': 'WILLIAMS REALTY INVESTMENT',
 'photo': None,
 'phone': None,
 'slogan': None,
 'show_realtor_logo': False,
 'link': None,
 'accent_color': None}}},
 'prop_status': 'for_sale',
 'price': 349000,
 'baths_full': 3,
 'baths': 3,
 'beds': 5,
 'agents': [{'primary': True,
 'advertiser_id': '1156950',
 'id': '1156950',
 'photo': None,
 'name': 'BURNELL WILLIAMS'}],
 'office': {'id': '2eaac57140bdf7ce46bdbfef8aa286c3',
 'name': 'WILLIAMS REALTY INVESTMENT'},
 'last_update': '2020-05-13T12:16:58Z',
 'client_display_flags': {'presentation_status': 'for_sale',
 'is_showcase': False,
 'lead_form_phone_required': True,
 'price_change': 0,
 'is_co_broke_email': True,
 'has_open_house': False,
 'is_co_broke_phone': False,
 'is_new_listing': True,
 'is_new_plan': False,
 'is_turbo': False,
 'is_office_standard_listing': False,
 'suppress_map_pin': False,
 'show_contact_a_lender_in_lead_form': True,
 'show_veterans_united_in_lead_form': True,
 'is_showcase_choice_enabled': False},
 'lead_forms': {'form': {'name': {'required': True,
 'minimum_character_count': 1},
 'email': {'required': True, 'minimum_character_count': 5},
 'phone': {'required': True,
 'minimum_character_count': 10,
 'maximum_character_count': 11},
 'message': {'required': False, 'minimum_character_count': 0},
 'show': True},
 'show_agent': False,
 'show_broker': False,
 'show_builder': False,
 'show_contact_a_lender': True,
 'show_veterans_united': True,
 'form_type': 'classic',
 'lead_type': 'co_broke',
 'is_lcm_enabled': False,
 'flip_the_market_enabled': False,
 'show_text_leads': True,
 'cashback_enabled': False},
 'photo_count': 1,
 'thumbnail': 'https://ap.rdcpix.com/bd26ed9d60d86373ebcec9bbd1498339l-m998730388x.jpg',
 'page_no': 1,
 'rank': 1,
 'list_tracking': 'type|property|data|prop_id|5185136719|list_id|2916184785|page|rank|list_branding|listing_agent|listing_office|advertiser_id|agent|property_status|product_code|advantage_code^1|1|0|1|OSPI|35T|9HS|1^^$0|1|2|$3|4|5|6|7|H|8|I|9|$A|J|B|K]|C|$D|L]|E|M|F|N|G|O]]',
 'lot_size': {'size': 2614, 'units': 'sqft'},
 'mls': {'name': 'GardenState',
 'id': '3632342',
 'plan_id': None,
 'abbreviation': 'GSNJ',
 'type': 'mls'}},
 {'property_id': 'M6481212784',
 'listing_id': '2916115469',
 'rdc_web_url': 'https://www.realtor.com/realestateandhomes-detail/80-Marne-St_Newark_NJ_07105_M64812-12784',
 'prop_type': 'single_family',
 'address': {'city': 'Newark',
 'line': '80 Marne St',
 

In [12]:
df_properties_for_sale_raw = process_list_for_sale_response(response_json=property_list_for_sale_response)
df_properties_for_sale_raw.head()

,property_id,listing_id,rdc_web_url,prop_type,address,branding,prop_status,price,baths_full,baths,beds,agents,office,last_update,client_display_flags,lead_forms,photo_count,thumbnail,page_no,rank,list_tracking,lot_size,mls,baths_half,prop_sub_type,building_size,virtual_tour
0,M5185136719,2916184785,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'Newark', 'line': '273 Lake St', 'pos...",{'listing_office': {'list_item': {'name': 'WIL...,for_sale,349000,3,3,5,"[{'primary': True, 'advertiser_id': '1156950',...","{'id': '2eaac57140bdf7ce46bdbfef8aa286c3', 'na...",2020-05-13T12:16:58Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",1,https://ap.rdcpix.com/bd26ed9d60d86373ebcec9bb...,1,1,type|property|data|prop_id|5185136719|list_id|...,"{'size': 2614, 'units': 'sqft'}","{'name': 'GardenState', 'id': '3632342', 'plan...",NaN,NaN,NaN,NaN
1,M6481212784,2916115469,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'Newark', 'line': '80 Marne St', 'pos...",{'listing_office': {'list_item': {'name': 'Ros...,for_sale,299000,2,2,3,"[{'primary': True, 'advertiser_id': '3176772',...","{'id': '4fbc177a7e9ca82486555eeba57fae54', 'na...",2020-05-11T14:28:23Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",0,NaN,1,2,type|property|data|prop_id|6481212784|list_id|...,NaN,"{'name': 'NewJerseyMLS', 'id': '20016255', 'pl...",NaN,NaN,NaN,NaN
2,M5302613491,2916080620,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'Newark', 'line': '196 Brookdale Ave'...",{'listing_office': {'list_item': {'name': 'RE/...,for_sale,259900,1,2,3,"[{'primary': True, 'advertiser_id': '1148457',...","{'id': '239b2fbc79596efc32ae7374aaab2201', 'na...",2020-05-09T20:06:18Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",24,https://ap.rdcpix.com/7f35aeddd6664d349fb1a578...,1,3,type|property|data|prop_id|5302613491|list_id|...,"{'size': 3920, 'units': 'sqft'}","{'name': 'GardenState', 'id': '3631740', 'plan...",1,NaN,NaN,NaN
3,M9838958461,2916078293,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'Newark', 'line': '26.5 Joseph St', '...",{'listing_office': {'list_item': {'name': 'RE/...,for_sale,229900,2,2,2,"[{'primary': True, 'advertiser_id': '2063678',...","{'id': '659b383b892f42ab6a4d8850d1e69fb7', 'na...",2020-05-09T17:03:14Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",21,https://ap.rdcpix.com/e4e250a28eaf74e5c4f95646...,1,4,type|property|data|prop_id|9838958461|list_id|...,"{'size': 1680, 'units': 'sqft'}","{'name': 'GardenState', 'id': '3631716', 'plan...",NaN,NaN,NaN,NaN
4,M5587229941,2916049258,https://www.realtor.com/realestateandhomes-det...,single_family,"{'city': 'Newark', 'line': '72 Nairn Pl', 'pos...",{'listing_office': {'list_item': {'name': 'Ora...,for_sale,375000,3,4,6,"[{'primary': True, 'photo': None, 'name': 'Eya...","{'id': 'e3e593a538abac3b1e8736bda2442671', 'na...",2020-05-08T15:00:23Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",25,https://ap.rdcpix.com/327de494b3118962703435d4...,1,5,type|property|data|prop_id|5587229941|list_id|...,NaN,"{'name': 'NewJerseyMLS', 'id': '20015991', 'pl...",1,NaN,NaN,NaN


In [13]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/detail"

querystring = {"property_id":"M5185136719"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': realtor_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

response.json()

Out[17]: {'meta': {'build': '3.23.91',
 'schema': 'core.3',
 'tracking': 'type|meta|data|resource_type|property_detail|query|client_id|rdc_mobile_native,13.3.0.53|schema|core.3|tag_version|v2^^^$0|1|2|$3|4|5|$6|7|8|9|A|B]]]',
 'returned_rows': 1,
 'matching_rows': 1,
 'tracking_params': {'ldpPropertyStatus': 'ldp:for_sale',
 'pageType': 'ldp',
 'leadDelivery': 'co_broke',
 'leadEnhancements': 'classic,rcm',
 'listingActivity': 'active',
 'productType': 'core.agent',
 'propertyStatus': 'for_sale',
 'listingId': '2916184785',
 'rentalDataSource': 'unknown',
 'advertiserIdAgent': '1156950',
 'communityId': 'unknown',
 'mprId': '5185136719',
 'listingMls': 'GSNJ',
 'planId': 'unknown',
 'subId': 'unknown',
 'city': 'Newark',
 'neighborhood': 'Seventh Avenue',
 'state': 'NJ',
 'zip': '07104',
 'listingBaths': '3',
 'listingBeds': '5',
 'listingSqFt': 'unknown',
 'listingEnhancements': 'broker-phone-btm',
 'listingPrice': '349000',
 'photoCount': '1',
 'propertyType': 'ldp:single_family',
 'version': '1.0'}},
 'properties': [{'property_id': 'M5185136719',
 'prop_status': 'for_sale',
 'listing_id': '2916184785',
 'prop_type': 'single_family',
 'suppression_flags': ['suppress_foreclosure'],
 'list_date': '2020-05-13T16:21:44Z',
 'hoa_fee': 0,
 'hoa_historic_fee': None,
 'last_update': '2020-05-13T12:16:58Z',
 'broker': {'name': 'WILLIAMS REALTY INVESTMENT',
 'phone1': {'number': '(973) 243-1700', 'type': 'broker'}},
 'year_built': 1909,
 'listing_status': 'Active',
 'beds': 5,
 'description': 'Great value for your money. Possible mother/daughter living. Across from park. Inside completely renovated. ?UST SEE!!!',
 'baths_full': 3,
 'stories': 2,
 'schools': [{'nces_id': '341134002244',
 'id': '0748590501',
 'greatschools_id': '3401335',
 'name': 'Branch Brook School',
 'education_levels': ['elementary'],
 'funding_type': 'public',
 'lat': 40.7611,
 'lon': -74.1763,
 'student_count': 151,
 'student_teacher_ratio': 12.8,
 'location': {'postal_code': '07104',
 'state': 'NJ',
 'county': 'Essex',
 'city': 'Newark',
 'street': '228 Ridge Street'},
 'phone': '(973) 268-5220',
 'distance_in_miles': 0.2,
 'grades': {'range': {'low': 'PK', 'high': '4'}},
 'relevance': 'assigned',
 'ratings': {'great_schools_rating': 6, 'parent_rating': 5}},
 {'nces_id': '341134002280',
 'id': '0748590731',
 'greatschools_id': '3401352',
 'name': 'Dr. William H. Horton Elementary School',
 'education_levels': ['elementary', 'middle'],
 'funding_type': 'public',
 'lat': 40.7598,
 'lon': -74.187,
 'student_count': 742,
 'student_teacher_ratio': 17.4,
 'location': {'postal_code': '07107',
 'state': 'NJ',
 'county': 'Essex',
 'city': 'Newark',
 'street': '291 North 7th Street'},
 'phone': '(973) 268-5260',
 'distance_in_miles': 0.4,
 'grades': {'range': {'low': 'PK', 'high': '8'}},
 'relevance': 'assigned',
 'ratings': {'great_schools_rating': 2, 'parent_rating': 2}},
 {'nces_id': '341134002190',
 'id': '0748590231',
 'greatschools_id': '3401317',
 'name': 'Barringer High School',
 'education_levels': ['high'],
 'funding_type': 'public',
 'lat': 40.7562,
 'lon': -74.1798,
 'student_count': 723,
 'student_teacher_ratio': 12.5,
 'location': {'postal_code': '07104',
 'state': 'NJ',
 'county': 'Essex',
 'city': 'Newark',
 'street': '90 Parker Street'},
 'phone': '(973) 268-5125',
 'distance_in_miles': 0.3,
 'grades': {'range': {'low': '9', 'high': '12'}},
 'relevance': 'assigned',
 'ratings': {'great_schools_rating': 1, 'parent_rating': 3}},
 {'nces_id': '341134002278',
 'id': '0748590721',
 'greatschools_id': '3401351',
 'name': 'Franklin Elementary School',
 'education_levels': ['elementary'],
 'funding_type': 'public',
 'lat': 40.7553,
 'lon': -74.1752,
 'student_count': 504,
 'student_teacher_ratio': 12.2,
 'location': {'postal_code': '07104',
 'state': 'NJ',
 'county': 'Essex',
 'city': 'Newark',
 'street': '42 Park Avenue'},
 'phone': '(973) 268-5250',
 'distance_in_miles': 0.4,
 'grades': {'range': {'low': 'PK', 'high': '4'}},
 'relevance': 'nearby',
 'ratings': 

# End Notebook